# 探索性数据分析

## 导入

导入必备的库：

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import pyarrow.feather as feather
from IPython import display
display.set_matplotlib_formats("svg")

C:\Users\wangjun\AppData\Local\Temp\ipykernel_3796\2975487684.py:8: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  display.set_matplotlib_formats("svg")


这里有一个warming，指出来了一些内置API的修改。

接着开始导入数据集：

In [9]:
# 打开压缩文件
with zipfile.ZipFile('./../datasets/house_sales.zip', 'r') as zip_ref:
    # 获取压缩文件中的文件列表
    file_list = zip_ref.namelist()
    
    # 假设压缩文件中只有一个Feather文件
    if len(file_list) == 1 and file_list[0].endswith('.ftr'):
        with zip_ref.open(file_list[0]) as file:
            data = feather.read_feather(file)
    else:
        raise ValueError("压缩文件中包含多个文件或没有Feather文件")

因为这是一个raw data，所以数据集里面存在着各种乱七八糟的东西。

In [10]:
data.shape

(164944, 1789)

数据集一共有16.5万个左右的样本，并且有1789个特征数量。

In [11]:
data.head()

,Id,Address,Sold Price,Sold On,Summary,Type,Year built,Heating,Cooling,Parking,...,Well Disclosure,remodeled,DOH2,SerialX,Full Baths,Tax Legal Lot Number,Tax Legal Block Number,Tax Legal Tract Number,Building Name,Zip
0,2080183300,"11205 Monterey,","$2,000,000",01/31/20,"11205 Monterey, San Martin, CA 95046 is a sing...",SingleFamily,No Data,No Data,No Data,0 spaces,...,None,None,None,None,None,None,None,None,None,95046
1,20926300,"5281 Castle Rd,","$2,100,000",02/25/21,Spectacular Mountain and incredible L.A. City ...,SingleFamily,1951,Central,"Central Air, Dual","Driveway, Driveway - Brick",...,None,None,None,None,None,None,None,None,None,91011
2,19595300,"3581 Butcher Dr,","$1,125,000",11/06/19,Eichler Style home! with Santa Clara High! in ...,SingleFamily,1954,Central Forced Air - Gas,Central AC,"Garage, Garage - Attached, Covered",...,None,None,None,None,None,None,None,None,None,95051
3,300472200,"2021 N Milpitas Blvd,","$36,250,000",10/02/20,"2021 N Milpitas Blvd, Milpitas, CA 95035 is a ...",Apartment,1989,Other,No Data,"Mixed, Covered",...,None,None,None,None,None,None,None,None,None,95035
4,2074492000,"LOT 4 Tool Box Spring Rd,","$140,000",10/19/20,Beautiful level lot dotted with pine trees ro...,VacantLand,No Data,No Data,No Data,0 spaces,...,None,None,None,None,None,None,None,None,None,92561


 可以发现数据中存在着大量的列都是缺少数据的，呈现出了None的情况。

数据处理首先需要考虑缺失值的情况，如果数据中有着大量的数据是缺失的，那么这里就将这个列删除，这里可以设置一个阈值，李沐老师给出来的是30%。

In [12]:
null_sum = data.isnull().sum()
null_sum

Id                             0
Address                        0
Sold Price                    85
Sold On                        0
Summary                     3117
                           ...  
Tax Legal Lot Number      164943
Tax Legal Block Number    164943
Tax Legal Tract Number    164943
Building Name             164943
Zip                            0
Length: 1789, dtype: int64

isnull对每个列的每个元素判断是否为None：

In [13]:
data.isnull()

,Id,Address,Sold Price,Sold On,Summary,Type,Year built,Heating,Cooling,Parking,...,Well Disclosure,remodeled,DOH2,SerialX,Full Baths,Tax Legal Lot Number,Tax Legal Block Number,Tax Legal Tract Number,Building Name,Zip
0,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,False
1,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,False
2,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,False
3,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,False
4,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164939,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,False
164940,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,False
164941,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,False
164942,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,False


In [14]:
data.columns[null_sum < len(data) * 0.3]

Index(['Id', 'Address', 'Sold Price', 'Sold On', 'Summary', 'Type',
       'Year built', 'Heating', 'Cooling', 'Parking', 'Bedrooms', 'Bathrooms',
       'Total interior livable area', 'Total spaces', 'Garage spaces',
       'Home type', 'Region', 'Elementary School', 'Elementary School Score',
       'Elementary School Distance', 'High School', 'High School Score',
       'High School Distance', 'Heating features', 'Parking features',
       'Lot size', 'Parcel number', 'Tax assessed value', 'Annual tax amount',
       'Listed On', 'Listed Price', 'Zip'],
      dtype='object')

这些是会被保存的列的名称。

接着删除这些缺失值占比特别高的列：

In [15]:
data.drop(columns = data.columns[null_sum > len(data) * 0.3], inplace = True)

接着查看一下删除之后的数据集的大小：

In [17]:
data.shape

(164944, 32)

可以发现比较有用的其实就只有32个特征。

接着查看一下这些特征的格式：

In [18]:
data.dtypes

Id                             object
Address                        object
Sold Price                     object
Sold On                        object
Summary                        object
Type                           object
Year built                     object
Heating                        object
Cooling                        object
Parking                        object
Bedrooms                       object
Bathrooms                      object
Total interior livable area    object
Total spaces                   object
Garage spaces                  object
Home type                      object
Region                         object
Elementary School              object
Elementary School Score        object
Elementary School Distance     object
High School                    object
High School Score              object
High School Distance           object
Heating features               object
Parking features               object
Lot size                       object
Parcel numbe

object的话其实就是本文格式表示的内容，但是有一些特征不应该是object，而应该是数值类型或者date类型等等。

In [24]:
currency = ['Sold Price', 'Listed Price', 'Tax assessed value', 'Annual tax amount']

for c in currency:
    data[c] = data[c].replace(
        r'[$, -]', '', regex = True).replace(
            r'^\s*$', np.nan, regex = True).astype(float)